<a href="https://colab.research.google.com/github/doubleblindreview2/jbr_video_mining/blob/master/video_mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Load All Models**
Requires no additional user input


# **2. Mount Drive and get User Input**
Please provide a Google Drive Account, which includes 
- Video files for video mining
- Folder for video mining results _(can be empty)_ 
- Folder for log file _(can be  empty)_


## **3. Run Video Mining**
Requires no additional user input